In [4]:
import pandas as pd

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/mike/.gcp-keys/pokemechs-84c9b2aac2f8.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/mike/.gcp-keys/pokemechs-84c9b2aac2f8.json


In [3]:

from typing import List, Optional

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel



In [4]:
def embed_text(
    texts: List[str],
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "text-embedding-004",
    dimensionality: Optional[int] = 256,
) -> List[List[float]]:
    """Embeds texts with a pre-trained, foundational model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
    embeddings = model.get_embeddings(inputs, **kwargs)
    return [embedding.values for embedding in embeddings]


In [5]:

filename = './manuals/Subaru-outback-starter-and-generater/Subaru_Legacy_Outback_starter-generater.md'
with open(filename, 'r') as fd:
    text = fd.read()



In [6]:
split_texts = text.split('\n## ')

In [7]:

split_text_with_identifier = []

for i in split_texts:
    text_identifier = "Alternator/Starter"
    modified_text = f"## {text_identifier}\n{i}"
    split_text_with_identifier.append(modified_text)


In [8]:
embedded_test = embed_text([text])

In [9]:
import json


output_filename = './manuals/embedded/Subaru_Outback.json'

with open(output_filename, 'w') as f:
    json.dump([list(emb) for emb in embedded_test], f) 


In [10]:
import psycopg2

In [11]:
psycopg2.connect

<function psycopg2.connect(dsn=None, connection_factory=None, cursor_factory=None, **kwargs)>

In [13]:
con =psycopg2.connect("user=postgres password=postgres host=127.0.0.1 port=64322 dbname=postgres")



In [ ]:
yearmin = 2010
yearmax = 2013

for i in split_text_with_identifier:
    embedded_text = embed_text([i])
    with con.cursor() as cur:
        cur.execute("insert into subaru (name, yearmin, yearmax, embedding, text) values ('Legacy-Outback starter/generater', %s, %s, %s, %s)", (yearmin, yearmax, embedded_text[0], i))
        con.commit()


In [32]:

def search(query, topk=3):
    try:
        dimensionality: Optional[int] = 256
        model = TextEmbeddingModel.from_pretrained("text-embedding-004") 
        query_embedding = model.get_embeddings([TextEmbeddingInput(query, "RETRIEVAL_QUERY")], output_dimensionality=dimensionality)[0].values
        print(len(query_embedding))

        with con.cursor() as cur:

            cur.execute(
                """
                SELECT id, name, text, 1 - (embedding <-> %s::vector(256)) AS similarity 
                FROM subaru
                ORDER BY similarity DESC
                LIMIT %s
                """,
                (query_embedding, topk)
            )
            results = cur.fetchall()
    except Exception as e:
        print(f"Error executing query: {e}")
        con.rollback()
        return []
    else:
        con.commit()
    finally:
        cur.close() 

    return results

In [40]:
from vertexai.generative_models import GenerativeModel

model = GenerativeModel(model_name="gemini-1.5-flash-001")

vehicle_make = 'subaru'
vehicle_model = 'outback'
vehicle_year = '2012'

def generate_response(query):
    search_results = search(query)

    prompt = f"""
    You are an expert mechanic specializing in {vehicle_year} {vehicle_make} {vehicle_model}'s.

    You have access to relevant sections from the official service manual for this vehicle. Use this information to provide a clear, concise, and accurate answer to the user's question.

    **Question:** {query}

    **Relevant Manual Sections:**
    ---
    {search_results[0][2]}
    ---
    {search_results[1][2]}
    ---
    {search_results[2][2]}
    ---

    **Important considersations**
    - Provide a clear, concise, and accurate answer to the user's question.
    - Exclude any irrelevant information or text formalities.
    - Be as breif as possible without sacrificing clarity.
    """


    response = model.generate_content(prompt)
    return response.text

user_question = "what are the steps to remove the starter motor"
response = generate_response(user_question)
print(response)


256
1. **Disconnect the battery.**
2. **Remove the air intake boot.** This will involve loosening clamps and moving the boot out of the way.  The specific steps depend on your engine size (2.5L or 3.6L). Refer to the relevant images and instructions in the manual. 
3. **Disconnect the electrical connector to the starter motor.** This is usually located on the side of the starter. 
4. **Remove the starter motor mounting bolts.** There are usually two or three bolts securing the starter to the transmission bell housing. 
5. **Carefully remove the starter motor.** You may need to wiggle it slightly to disengage it from the flywheel. 

